In [ ]:
import numpy as np
import json
import os

def generate_nodes(N, space_size=400, filename="nodes.json", initial_energy=100.0):
    """
    Sinh N node cảm biến ngẫu nhiên trong không gian 3D với residual energy
    
    N: số lượng node
    space_size: kích thước không gian
    filename: tên file lưu
    initial_energy: năng lượng ban đầu (E0)
    """
    
    np.random.seed(0)  # để kết quả lặp lại
    node_positions = np.random.rand(N, 3) * space_size
    
    # Sinh residual energy ngẫu nhiên (50-100% của initial energy)
    np.random.seed(42)  # Seed khác để energy không phụ thuộc vào vị trí
    residual_energies = np.random.uniform(0.5 * initial_energy, initial_energy, N)

    data = []
    for i in range(N):
        data.append({
            "id": i,
            "x": float(node_positions[i][0]),
            "y": float(node_positions[i][1]),
            "z": float(node_positions[i][2]),
            "residual_energy": float(residual_energies[i]),
            "initial_energy": float(initial_energy)
        })

    
    os.makedirs("input_data", exist_ok=True)
    filepath = f"input_data/{filename}"
    with open(filepath, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Đã tạo file {filepath} chứa {N} node với residual energy.")
    print(f"Energy range: {residual_energies.min():.1f} - {residual_energies.max():.1f}")

# Sinh dữ liệu với energy information
for N in [20, 100, 200, 500, 1000]:
    generate_nodes(N, space_size=100, filename=f"nodes_{N}.json", initial_energy=100.0)

Đã tạo file input_data/nodes_20.json chứa 20 node.
Đã tạo file input_data/nodes_100.json chứa 100 node.
Đã tạo file input_data/nodes_200.json chứa 200 node.
Đã tạo file input_data/nodes_500.json chứa 500 node.
Đã tạo file input_data/nodes_1000.json chứa 1000 node.


In [1]:
import numpy as np

def calculate_number_clusters(nodes, base_station=(0, 0, 0), space_size=400):
    """
    Tính số cụm optimal theo công thức từ bài báo:
    K = √( / πd_tobs)
    
    nodes: tọa độ 3D của các node
    base_station: tọa độ base station (mặc định tại gốc tọa độ)
    """
    N = len(nodes)
    base_pos = np.array(base_station)
    
    # Tính khoảng cách trung bình từ các node đến base station
    distances = np.linalg.norm(nodes - base_pos, axis=1)
    d_tobs = np.mean(distances)
    
    # Áp dụng công thức K = √(N*L / πd_tobs)
    K_optimal = np.sqrt(N * space_size / (np.pi * d_tobs))
    K_optimal = max(1, int(np.round(K_optimal)))  # Đảm bảo K >= 1 và là số nguyên
    
    print(f"N = {N}, d_tobs = {d_tobs:.2f}, K_optimal = {K_optimal}")
    return K_optimal

In [2]:
import numpy as np
from sklearn.cluster import KMeans

def cluster_split(nodes, node_ids, node_data=None, r_sen=100, R=20, max_depth=10, depth=0):
    """
    Hàm phân cụm lặp theo Algorithm 1
    nodes: tọa độ 3D của các node
    node_ids: list id tương ứng của các node
    node_data: dictionary chứa thông tin đầy đủ của nodes (bao gồm energy)
    r_sen: bán kính truyền tải tối đa của node, giả sử là 100m
    R: số lượng node tối đa trong 1 cụm, cho là 20
    max_depth: độ sâu đệ quy tối đa
    """
    
    center = np.mean(nodes, axis=0)
    dists = np.linalg.norm(nodes - center, axis=1) # khoảng cách từ tâm đến các node
    if (len(nodes) <= R and np.all(dists <= r_sen)) or depth >= max_depth:
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]

    # Kmeans với k=2 để chia cụm
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    labels = kmeans.fit_predict(nodes)

    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        
        # Tạo sub_node_data cho cluster con
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, max_depth, depth + 1)

    return clusters


def choose_cluster_head(cluster, node_data_dict):
    """
    Chọn cluster head theo công thức desirableness factor:
    Q = d_tocenter^(E0/E_current)
    
    Node có Q MAX sẽ được chọn làm cluster head
    
    cluster: thông tin cluster
    node_data_dict: dictionary chứa thông tin đầy đủ của tất cả nodes
    """
    nodes = cluster["nodes"]
    center = cluster["center"]
    node_ids = cluster["node_ids"]

    # Tính khoảng cách từ mỗi node đến tâm cụm
    dists_to_center = np.linalg.norm(nodes - center, axis=1)
    
    max_Q = -1
    best_cluster_head = node_ids[0]  # Default fallback
    
    print(f"\n=== Chọn Cluster Head cho cụm có {len(node_ids)} nodes ===")
    
    for i, node_id in enumerate(node_ids):
        if node_id in node_data_dict:
            node_info = node_data_dict[node_id]
            
            # Lấy thông tin energy
            E_current = node_info.get('residual_energy', 100.0)
            E0 = node_info.get('initial_energy', 100.0)
            
            # Tính desirableness factor: Q = d_tocenter^(E0/E_current)
            d_tocenter = dists_to_center[i]
            
            # Tránh chia cho 0 và overflow
            if E_current <= 0:
                E_current = 0.1  # Năng lượng rất thấp
            
            energy_ratio = E0 / E_current
            
            # Tính Q - node có Q MAX sẽ được chọn
            Q = d_tocenter ** energy_ratio
            
            print(f"  Node {node_id}: d={d_tocenter:.2f}, E_cur={E_current:.1f}, "
                  f"E0={E0:.1f}, ratio={energy_ratio:.2f}, Q={Q:.3f}")
            
            # Chọn node có Q lớn nhất
            if Q > max_Q:
                max_Q = Q
                best_cluster_head = node_id
        else:
            # Fallback: chọn node gần tâm nhất nếu không có thông tin energy
            print(f"  Node {node_id}: No energy data, using distance only")
            if i == 0 or dists_to_center[i] < dists_to_center[node_ids.index(best_cluster_head)]:
                best_cluster_head = node_id
    
    print(f"Chọn Node {best_cluster_head} làm Cluster Head (Q_max = {max_Q:.3f})")
    return best_cluster_head

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

input_folder = "D:\\Year 4\\tiến hóa\\project\\UWSN_greedy\\input_data"
output_folder = "D:\\Year 4\\tiến hóa\\project\\UWSN_greedy\\output_data_kmeans"
os.makedirs(output_folder, exist_ok=True)
draw_folder = "draw_output_kmeans"
os.makedirs(draw_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.startswith("nodes_") and filename.endswith(".json"):
        # Lấy số lượng node từ tên file
        number_nodes = filename.split("_")[1].split(".")[0]
        # Đọc dữ liệu từ file input
        with open(os.path.join(input_folder, filename), "r") as f:
            data = json.load(f)
        node_positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
        node_ids = [d["id"] for d in data]
        # Phân cụm
        k = calculate_number_clusters(node_positions, base_station=(0, 0, 0))
        print(f"Số cụm: {k}")
        clusters_raw = cluster_split(node_positions, node_ids, R=int(number_nodes)//k)
        # Tạo output
        clusters_output = {}
        for i, c in enumerate(clusters_raw):
            ch = choose_cluster_head(c)
            clusters_output[i] = {
                "nodes": c["node_ids"],
                "center": tuple(np.round(c["center"], 2)),
                "cluster_head": int(ch)
            }
        # Xuất ra file
        out_path = os.path.join(output_folder, f"nodes_{number_nodes}.json")
        with open(out_path, "w") as f:
            json.dump(clusters_output, f, indent=4)
        print(f"Đã xuất file {out_path}")
        # Vẽ và lưu hình
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        colors = plt.cm.get_cmap('tab10', len(clusters_output))
        for cid, info in clusters_output.items():
            nodes = np.array([node_positions[nid] for nid in info['nodes']])
            ax.scatter(nodes[:, 0], nodes[:, 1], nodes[:, 2],
                        label=f'Cụm {cid}',
                        color=colors(cid))
            ch_pos = node_positions[info['cluster_head']]
            ax.scatter(ch_pos[0], ch_pos[1], ch_pos[2],
                        color=colors(cid),
                        marker='*', s=80, edgecolor='k')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.legend()
        plt.title(f'Phân cụm các node cảm biến ({number_nodes} node)')
        ax.view_init(elev=30, azim=30)
        plt.tight_layout()
        draw_path = os.path.join(draw_folder, f"nodes_{number_nodes}.png")
        plt.savefig(draw_path)
        plt.close(fig)
        print(f"Đã lưu hình vẽ {draw_path}")

N = 100, d_tobs = 96.71, K_optimal = 11
Số cụm: 11


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subp

TypeError: choose_cluster_head() missing 1 required positional argument: 'node_data_dict'